In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import pyLDAvis.sklearn

import sys
import os
import logging
import re
import csv
import json
import smart_open
import codecs
import nltk
import time
import pickle
import numpy as np
from collections import OrderedDict

from text_process.tweets_text_processor import TextProcessor

In [4]:
def load_model(model_file, tweets):
    pipeline = pickle.load(open(model_file, "rb"))
    model = pipeline.transform(tweets)
    lda = pipeline.named_steps['lda']
    vect = pipeline.named_steps['vect']
    return [pipeline, model, lda, vect]

In [5]:
def get_topic_word_dist(topics, feature_names, num_words=100):
    word_dists = []  # [{word1: weight1, word2: weight2, word3: weight3....}, {word1, weight1, word2: weight2....}...]
    for topic_idx, topic in enumerate(topics):
        feature_list = OrderedDict()  # {word1: weight, word2: weight ...}
        for i in topic.argsort()[:-num_words - 1:-1]:
            feature = feature_names[i]
            weight = round(topic[i], 2)
            feature_list[feature] = weight

        word_dists.append(feature_list)
    return word_dists

In [6]:
def get_topic_dist(components, feature_names, model):
    average = np.average(np.array(model), axis=0)
    top_topics = average.argsort()[::-1]
    top_component = [components[i] for i in top_topics]
    top_topics_words = get_topic_word_dist(top_component, feature_names)

    top_topics = np.array(list(map(lambda x:x+1, top_topics)))
    result = {"avg": average, "top_topics": top_topics, "top_component": top_component, "top_topics_words":
              top_topics_words}
    return result

In [7]:
def consine_similarirty(vec1, vec2):
    numerator = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    return numerator / (norm_vec1 * norm_vec2)

In [8]:
def compare_topics(pipeline, model, num_bf_doc):
    # TODO find the right threshold
    THRESDHOLD = 20

    components = pipeline._final_estimator.components_
    feature_names = pipeline.named_steps['vect'].get_feature_names()

    # overall topic distributions
    model_t = model
    result_t = get_topic_dist(components, feature_names, model_t)

    model_bf = model[:num_bf_doc]
    result_bf = get_topic_dist(components, feature_names, model_bf)

    top_topics_bf = result_bf['top_topics'][:THRESDHOLD]

    model_af = model[num_bf_doc:]
    result_af = get_topic_dist(components, feature_names, model_af)
    top_topics_af = result_af['top_topics'][:THRESDHOLD]

    print("Topic Distribution Similarity", consine_similarirty(result_bf['avg'], result_af['avg']))
    print("Common topic", list(top_topics_bf | top_topics_af))
    unique_topic = list(set(top_topics_bf).symmetric_difference(set(top_topics_af)))
    print("Unique Topic", unique_topic)
    bf_unique = list(filter(lambda x: x in unique_topic, top_topics_bf))
    af_unique = list(filter(lambda x: x in unique_topic, top_topics_af))
    bf_unique_diff = [str(result_bf["avg"][x-1])for x in bf_unique]
    af_unique_diff = [str(result_af["avg"][x-1])for x in af_unique]
    print("BF unique topic", bf_unique, bf_unique_diff)
    print("AF unique topic", af_unique, af_unique_diff)

    print("\n\n")
    # TODO print unique topics for period

    return result_t, result_bf, result_af

In [9]:
def show_topic_dist(result, period, num_topics=20):
    print("{} topic distribution: \n".format(period))

    top_topics = result["top_topics"]
    print("top_topics: \n", ' | '.join([str(top_topics[i]) + " - " + str(result["avg"][top_topics[i]-1]) for i in range(num_topics)]))
    for i, words in enumerate([', '.join(list(dist.keys())[:15]) for dist in result["top_topics_words"][:num_topics]]):
        print("topic " + str(top_topics[i]) + " top words: \n", words)
    print("\n\n")

In [10]:
def show_topics(result_t, result_bf, result_af):
    # overall topic distributions
    show_topic_dist(result_t, "overall")

    show_topic_dist(result_bf, "before")

    show_topic_dist(result_af, "after")


In [11]:
def extract_tweet_with_period(incident, input_paths, output_path, metadata_path):
    """
    extract tweets before and after disaster start
    input_paths are two paths, geotagged and timeline folders
    """
    file_bf = os.path.join(output_path, "{}_tweets_bf.csv".format(incident))
    file_af = os.path.join(output_path, "{}_tweets_af.csv".format(incident))

    if os.path.isfile(file_bf) and os.path.isfile(file_af):
        logging.debug("Tweets already extracted")
        corpus = []
        files = [file_bf, file_af]
        for file in files:
            with open(file,'r') as file:
                lines = file.readlines()
                tweets = lines[1:]
                corpus.append(tweets)

            logging.debug(tweets[0:5])

    else:
        text_proc = TextProcessor(metadata_path, input_paths, output_path, model="topic")
        corpus = text_proc.process_tweets(incident)

    return corpus

In [12]:
input_paths = ["/Users/stellachoi/Box Sync/research_work/events_tweets/Event - 319 - Moore Tornado/geotagged_from_archive/",
               "/Users/stellachoi/Box Sync/research_work/events_tweets/Event - 319 - Moore Tornado/user_timelines/"]
incident_metadata_path = '/Users/stellachoi/Box Sync/research_work/disaster_analysis/incident_metadata.csv'
output_path = "/Users/stellachoi/Box Sync/research_work/disaster_analysis/data"
incident = 319
corpus = extract_tweet_with_period(incident, input_paths, output_path, incident_metadata_path)
tweets = corpus[0][:]
tweets.extend(corpus[1])
print("tweet_bf_len: {} tweet_af_len: {}".format(len(corpus[0]), len(corpus[1])))

tweet_bf_len: 435723 tweet_af_len: 769034


In [13]:
model_file = "backup/topic_pipeline_100_0305_1.p"
[pipeline, model, lda, vect] = load_model(model_file, tweets)
[result_t, result_bf, result_af] = compare_topics(pipeline, model, len(corpus[0]))
show_topics(result_t, result_bf, result_af)


Topic Distribution Similarity 0.998423624339
Common topic [98, 64, 73, 4, 73, 111, 111, 26, 70, 33, 115, 87, 94, 87, 111, 127, 90, 95, 100, 63]
Unique Topic [2, 11, 19, 96, 100, 45]
BF unique topic [11, 96, 45] ['0.0121278772737', '0.0118833073899', '0.0118066496072']
AF unique topic [19, 2, 100] ['0.0140305296143', '0.0127287698619', '0.0121966581746']



overall topic distribution: 

top_topics: 
 98 - 0.0220655046756 | 64 - 0.017818341449 | 73 - 0.0172755935108 | 4 - 0.0171673339216 | 9 - 0.0168327662759 | 47 - 0.0163389056468 | 68 - 0.0162061543709 | 26 - 0.0155795146073 | 70 - 0.0152073790755 | 33 - 0.0150172470159 | 19 - 0.0131830849531 | 99 - 0.0131816157404 | 82 - 0.0131725678466 | 5 - 0.0131366171254 | 90 - 0.0130080250889 | 79 - 0.0129834762354 | 94 - 0.0126127475065 | 2 - 0.0123904865884 | 55 - 0.0123878957868 | 25 - 0.0118983118106
topic 98 top words: 
 day, life, school, head, alreadi, hey, end, gone, left, spend, sign, thunder, fit, hurri, univers
topic 64 top words: 
 lo

In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, vect.transform(corpus[0]), vect, sort_topics=False, mds='tsne')
panel

/Users/stellachoi/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      0.845989        1       1 -0.200570  2.217279
1      1.175199        1       2 -0.367457  1.990864
2      1.049500        1       3 -0.337063  2.308699
3      1.949855        1       4  0.048355  3.303142
4      1.296477        1       5 -0.464390  2.155417
5      0.832149        1       6 -0.234777  2.203348
6      0.887339        1       7 -0.203280  2.215410
7      0.866285        1       8 -0.212324  2.213970
8      1.930894        1       9 -0.331559  1.055533
9      0.602314        1      10 -0.246690  2.195978
10     1.290485        1      11 -0.353154  1.986964
11     0.756114        1      12 -0.214785  2.211905
12     1.141807        1      13 -0.425191  2.263974
13     0.750973        1      14 -0.235859  2.202633
14     0.460971        1      15 -0.249478  2.195703
15     0.698626        1      16 -0.240718  2.199063
16     0.729821        1      17 -0.238684  2.200282
17     0.876953        1      18 -0.203460  2.215218
18     1.286478        1      19 -0.463649  2.042613
19     0.733210        1      20 -0.239199  2.199950
20     0.746924        1      21 -0.237881  2.200940
21     0.741434        1      22 -0.237227  2.201718
22     0.836962        1      23 -0.216572  2.210274
23     0.853613        1      24 -0.218250  2.208531
24     1.169984        1      25 -0.320678  2.005489
25     1.605954        1      26  0.608721  2.781900
26     1.130703        1      27 -0.418570  2.270319
27     0.630131        1      28 -0.245684  2.196468
28     0.733841        1      29 -0.242271  2.198188
29     0.833857        1      30 -0.208630  2.212600
...         ...      ...     ...       ...       ...
70     0.747993        1      71 -0.239817  2.199595
71     1.173089        1      72 -0.418708  2.271765
72     1.840525        1      73  0.383393  1.342322
73     1.071477        1      74 -0.267648  2.019428
74     1.070551        1      75 -0.279192  2.300027
75     1.141403        1      76 -0.230141  2.029091
76     1.134872        1      77 -0.306149  2.308994
77     0.496817        1      78 -0.248963  2.195477
78     1.359170        1      79 -0.470163  2.115618
79     0.832198        1      80 -0.219834  2.207340
80     0.606064        1      81 -0.247365  2.195767
81     1.291970        1      82 -0.474416  2.138258
82     0.947521        1      83 -0.179051  2.223756
83     1.098617        1      84 -0.253475  2.024811
84     0.480105        1      85 -0.249152  2.195470
85     1.027100        1      86 -0.186376  2.226461
86     1.187161        1      87 -0.453405  2.232197
87     0.701336        1      88 -0.243354  2.197566
88     0.477671        1      89 -0.249717  2.195213
89     1.325977        1      90 -0.478810  2.094607
90     0.953807        1      91 -0.205443  2.214341
91     0.732536        1      92 -0.242156  2.198232
92     0.892786        1      93 -0.218522  2.208330
93     1.403176        1      94 -0.464760  2.157844
94     1.138440        1      95 -0.274098  2.014638
95     1.237664        1      96 -0.316743  2.004030
96     1.058649        1      97 -0.262194  2.295008
97     2.603659        1      98 -1.235033  1.656097
98     1.352195        1      99 -0.471744  2.145902
99     0.943888        1     100 -0.449032  2.240589

[100 rows x 5 columns], topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
5093   Default  23123.000000        like  23123.000000  30.0000  30.0000
5224   Default  19864.000000        love  19864.000000  29.0000  29.0000
5169   Default  17432.000000         lol  17432.000000  28.0000  28.0000
8839   Default  14327.000000        time  14327.000000  27.0000  27.0000
2312   Default  16744.000000         day  16744.000000  26.0000  26.0000
4879   Default  14056.000000        know  14056.000000  25.0000  25.0000
3673   Def

In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, vect.transform(corpus[1]), vect, sort_topics=False, mds='tsne')
panel

/Users/stellachoi/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      0.889130        1       1 -0.200570  2.217279
1      1.241012        1       2 -0.367457  1.990864
2      1.119057        1       3 -0.337063  2.308699
3      1.983128        1       4  0.048355  3.303142
4      1.249916        1       5 -0.464390  2.155417
5      0.784577        1       6 -0.234777  2.203348
6      0.865871        1       7 -0.203280  2.215410
7      0.808554        1       8 -0.212324  2.213970
8      1.853559        1       9 -0.331559  1.055533
9      0.653497        1      10 -0.246690  2.195978
10     1.281619        1      11 -0.353154  1.986964
11     0.916194        1      12 -0.214785  2.211905
12     1.149144        1      13 -0.425191  2.263974
13     0.757996        1      14 -0.235859  2.202633
14     0.475527        1      15 -0.249478  2.195703
15     0.729241        1      16 -0.240718  2.199063
16     0.751991        1      17 -0.238684  2.200282
17     0.865463        1      18 -0.203460  2.215218
18     1.632751        1      19 -0.463649  2.042613
19     0.739438        1      20 -0.239199  2.199950
20     0.748714        1      21 -0.237881  2.200940
21     0.761211        1      22 -0.237227  2.201718
22     0.822874        1      23 -0.216572  2.210274
23     0.825849        1      24 -0.218250  2.208531
24     1.178856        1      25 -0.320678  2.005489
25     1.542066        1      26  0.608721  2.781900
26     1.149305        1      27 -0.418570  2.270319
27     0.644846        1      28 -0.245684  2.196468
28     0.708003        1      29 -0.242271  2.198188
29     0.876058        1      30 -0.208630  2.212600
...         ...      ...     ...       ...       ...
70     0.725697        1      71 -0.239817  2.199595
71     1.184134        1      72 -0.418708  2.271765
72     1.888447        1      73  0.383393  1.342322
73     1.084408        1      74 -0.267648  2.019428
74     1.022164        1      75 -0.279192  2.300027
75     1.276291        1      76 -0.230141  2.029091
76     1.053725        1      77 -0.306149  2.308994
77     0.507694        1      78 -0.248963  2.195477
78     1.362937        1      79 -0.470163  2.115618
79     0.806884        1      80 -0.219834  2.207340
80     0.602117        1      81 -0.247365  2.195767
81     1.315287        1      82 -0.474416  2.138258
82     0.977540        1      83 -0.179051  2.223756
83     1.109846        1      84 -0.253475  2.024811
84     0.479810        1      85 -0.249152  2.195470
85     0.918646        1      86 -0.186376  2.226461
86     1.230524        1      87 -0.453405  2.232197
87     0.695031        1      88 -0.243354  2.197566
88     0.485790        1      89 -0.249717  2.195213
89     1.357255        1      90 -0.478810  2.094607
90     0.892630        1      91 -0.205443  2.214341
91     0.700650        1      92 -0.242156  2.198232
92     0.855991        1      93 -0.218522  2.208330
93     1.294709        1      94 -0.464760  2.157844
94     1.133697        1      95 -0.274098  2.014638
95     1.199775        1      96 -0.316743  2.004030
96     0.996771        1      97 -0.262194  2.295008
97     2.278623        1      98 -1.235033  1.656097
98     1.292226        1      99 -0.471744  2.145902
99     1.377628        1     100 -0.449032  2.240589

[100 rows x 5 columns], topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
5093   Default  41184.000000        like  41184.000000  30.0000  30.0000
5224   Default  33421.000000        love  33421.000000  29.0000  29.0000
5169   Default  30988.000000         lol  30988.000000  28.0000  28.0000
8839   Default  24917.000000        time  24917.000000  27.0000  27.0000
4879   Default  25196.000000        know  25196.000000  26.0000  26.0000
2312   Default  25662.000000         day  25662.000000  25.0000  25.0000
3673   Def